# Mount source code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Check GPU specs

In [ ]:
!nvidia-smi

Wed Apr 10 20:49:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# path and scripts definition

In [ ]:
resnet_experiment_root_path = "/content/drive/MyDrive/CS5260_A6" # Change this according to the Google Drive folder name
resnet_experiment_requirements_path = f"{resnet_experiment_root_path}/requirements.txt"
resnet_experiment_model_path = f"{resnet_experiment_root_path}/model"
resnet_experiment_train_script_path = f"{resnet_experiment_root_path}/train.py"
resnet_experiment_eval_script_path = f"{resnet_experiment_root_path}/eval.py"
colossal_ai_run = "/usr/local/bin/colossalai run"

# Install dependencies

In [ ]:
pip_install_command = f"pip install -r {resnet_experiment_requirements_path}"
!{pip_install_command}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB

# Make sure colossalai is installed successfully

In [ ]:
!/usr/local/bin/colossalai check -i

#### Installation Report ####

------------ Environment ------------
Colossal-AI version: 0.3.6
PyTorch version: 2.2.1
System CUDA version: 12.2
CUDA version required by PyTorch: 12.1

Note:
1. The table above checks the versions of the libraries/tools in the current environment
2. If the System CUDA version is N/A, you can set the CUDA_HOME environment variable to locate it
3. If the CUDA version required by PyTorch is N/A, you probably did not install a CUDA-compatible PyTorch. This value is give by torch.version.cuda and you can go to https://pytorch.org/get-started/locally/ to download the correct version.

------------ CUDA Extensions AOT Compilation ------------
Found AOT CUDA Extension: ✓
PyTorch version used for AOT compilation: N/A
CUDA version used for AOT compilation: N/A

Note:
1. AOT (ahead-of-time) compilation of the CUDA kernels occurs during installation when the environment variable BUILD_EXT=1 is set
2. If AOT compilation is not enabled, stay calm as the CUDA kernels 

# Training

In [ ]:
# Number of GPUs available. Since we are in Google Colab which has only 1 GPU, set it to 1.
# Change this accordingly if running in other environment.
num_gpu = 1

# train with torch DDP with fp32
train_with_DDP_fp32_command = f"{colossal_ai_run} --nproc_per_node {num_gpu} {resnet_experiment_train_script_path} -c {resnet_experiment_model_path}/ckpt-fp32"
!{train_with_DDP_fp32_command}

# train with torch DDP with mixed precision training
train_with_DDP_fp16_command = f"{colossal_ai_run} --nproc_per_node {num_gpu} {resnet_experiment_train_script_path} -c {resnet_experiment_model_path}/ckpt-fp16 -p torch_ddp_fp16"
!{train_with_DDP_fp16_command}

# train with low level zero
train_with_low_level_zero_command = f"{colossal_ai_run} --nproc_per_node {num_gpu} {resnet_experiment_train_script_path} -c {resnet_experiment_model_path}/ckpt-low_level_zero -p low_level_zero"
!{train_with_low_level_zero_command}

/usr/local/lib/python3.10/dist-packages/colossalai/pipeline/schedule/_utils.py:19: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _register_pytree_node(OrderedDict, _odict_flatten, _odict_unflatten)
/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:254: UserWarning: <class 'collections.OrderedDict'> is already registered as pytree node. Overwriting the previous registration.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/colossalai/shardformer/layer/normalization.py:45: UserWarning: Please install apex from source (https://github.com/NVIDIA/apex) to use the fused layernorm kernel
  warnings.warn("Please install apex from source (https://github.com/NVIDIA/apex) to use the fused layernorm kernel")
/usr/local/lib/python3.10/dist-packages/colossalai/initialize.py:48: UserWarning: `config` is deprecated and will be removed soon.
  warnings.warn("`config` is deprecated and will be rem

# Evaluation and reproduce performance table

In [ ]:
import re
import pandas as pd

ddp_fp32_result_path = './ddp_fp32.txt'
ddp_fp16_result_path = './ddp_fp16.txt'
low_level_zero_result_path = './low_level_zero.txt'

eval_with_DDP_fp32_command = f"python {resnet_experiment_eval_script_path} -c {resnet_experiment_model_path}/ckpt-fp32 -e 80 > {ddp_fp32_result_path}"
!{eval_with_DDP_fp32_command}
eval_with_DDP_fp16_command = f"python {resnet_experiment_eval_script_path} -c {resnet_experiment_model_path}/ckpt-fp16 -e 80 > {ddp_fp16_result_path}"
!{eval_with_DDP_fp16_command}
eval_with_low_level_zero_command = f"python {resnet_experiment_eval_script_path} -c {resnet_experiment_model_path}/ckpt-low_level_zero -e 80 > {low_level_zero_result_path}"
!{eval_with_low_level_zero_command}

with open(ddp_fp32_result_path, 'r') as file:
    ddp_fp32_result_txt = file.read()

with open(ddp_fp16_result_path, 'r') as file:
    ddp_fp16_result_txt = file.read()

with open(low_level_zero_result_path, 'r') as file:
    low_level_zero_result_txt = file.read()

accuracy_pattern = r'[0-9]+\.[0-9]+'
match_ddp_fp32 = re.search(accuracy_pattern, ddp_fp32_result_txt).group()
match_ddp_fp16 = re.search(accuracy_pattern, ddp_fp16_result_txt).group()
match_low_level_zero = re.search(accuracy_pattern, low_level_zero_result_txt).group()

headers = ['Model', 'Booster DDP with FP32', 'Booster DDP with FP16', 'Booster Low Level Zero']
performances = ['ResNet-18', f'{match_ddp_fp32}%', f'{match_ddp_fp16}%', f'{match_low_level_zero}%']
reproduced_result = pd.DataFrame([performances], columns=headers).reset_index(drop=True)
reproduced_result.head()

,Model,Booster DDP with FP32,Booster DDP with FP16,Booster Low Level Zero
0,ResNet-18,84.54%,84.77%,84.78%
